# Automated Test Script

#### Importing required libraries

In [211]:
import numpy as np
import pandas as pd
import sys, os

import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

#### Locating our logs files in our working directory

In [212]:
LOGS_PATH = os.path.join(os.getcwd(), 'logs')

#### Replace this with the log file name that your looking to test

In [213]:
LOG_BARCODE = '2018-04-07_19-36'

#### Loading DeepEye & Tester log files into Pandas DataFrames

In [214]:
tester_log = pd.read_csv(os.path.join(LOGS_PATH, "[" + LOG_BARCODE + "]--DeepEye.csv"), index_col=0)
deepeye_log = pd.read_csv(os.path.join(LOGS_PATH, "[" + LOG_BARCODE + "]--Tester.csv"), index_col=0) 

## Compare the two dataframes and identify each feature where the `GROUND TRUTH` and the `PREDICTION` by DeepEye were different

In [215]:
results = pd.concat([tester_log.set_index('FRAME_ID'), deepeye_log.set_index('FRAME_ID')], 
                   axis='columns', keys=['GROUND TRUTH', 'PREDICTION'])

results = results.swaplevel(axis='columns')[tester_log.columns[1:]]
results.head()

PEDESTRIAN                VEHICLES                   BIKES  \
         GROUND TRUTH PREDICTION GROUND TRUTH PREDICTION GROUND TRUTH   
FRAME_ID                                                                
0                   1          1            1          1            0   
1                   0          1            1          1            0   
2                   0          1            1          1            0   
3                   1          1            1          1            0   
4                   0          1            1          1            0   

                       STOP_SIGN            TRAFFIC_LIGHT             \
         PREDICTION GROUND TRUTH PREDICTION  GROUND TRUTH PREDICTION   
FRAME_ID                                                               
0                 0            0          0             0          1   
1                 0            0          0             0          1   
2                 0            0          0             0          1   
3                 0            0          0             0          1   
4                 0            0          0             0          1   

             OFF_LANE               COLLISION             
         GROUND TRUTH PREDICTION GROUND TRUTH PREDICTION  
FRAME_ID                                                  
0                   0          0            0          0  
1                   0          0            0          0  
2                   1          0            0          0  
3                   1          1            0          0  
4                   0          0            0          0

#### Look at False Positive/Negative Predictions

In [216]:
log = (tester_log != deepeye_log).stack()
df = log[log]
df.index.names = ['FRAME_ID', 'FEATURE']
difference_loc = np.where(tester_log != deepeye_log)
ground_truth = tester_log.values[difference_loc]
prediction = deepeye_log.values[difference_loc]
wrong_predictions = pd.DataFrame({'GROUND TRUTH': ground_truth, 'PREDICTION': prediction}, index=df.index)

In [217]:
wrong_predictions.head()

GROUND TRUTH  PREDICTION
FRAME_ID FEATURE                                
0        TRAFFIC_LIGHT             0           1
1        PEDESTRIAN                0           1
         TRAFFIC_LIGHT             0           1
2        PEDESTRIAN                0           1
         TRAFFIC_LIGHT             0           1

### Calculate a score for each frame based on the number of false predictions

In [218]:
scores = wrong_predictions.groupby(['FRAME_ID']).size().reset_index(name='SCORE')
scores =  len(tester_log.columns) - scores['SCORE'] - 1

results['ACCURACY', 'SCORE'] = scores
results['ACCURACY', 'SCORE'].fillna(len(tester_log.columns) - 1, inplace=True) 
results['ACCURACY', 'SCORE'] = results['ACCURACY', 'SCORE'] / 7
results.style.format("{:.2%}", subset=pd.IndexSlice[:, pd.IndexSlice[:, 'SCORE']])

results.head()

PEDESTRIAN                VEHICLES                   BIKES  \
         GROUND TRUTH PREDICTION GROUND TRUTH PREDICTION GROUND TRUTH   
FRAME_ID                                                                
0                   1          1            1          1            0   
1                   0          1            1          1            0   
2                   0          1            1          1            0   
3                   1          1            1          1            0   
4                   0          1            1          1            0   

                       STOP_SIGN            TRAFFIC_LIGHT             \
         PREDICTION GROUND TRUTH PREDICTION  GROUND TRUTH PREDICTION   
FRAME_ID                                                               
0                 0            0          0             0          1   
1                 0            0          0             0          1   
2                 0            0          0             0          1   
3                 0            0          0             0          1   
4                 0            0          0             0          1   

             OFF_LANE               COLLISION             ACCURACY  
         GROUND TRUTH PREDICTION GROUND TRUTH PREDICTION     SCORE  
FRAME_ID                                                            
0                   0          0            0          0  0.857143  
1                   0          0            0          0  0.714286  
2                   1          0            0          0  0.571429  
3                   1          1            0          0  0.857143  
4                   0          0            0          0  0.714286

## Display some basic statistical analytics

### Overall Performance

In [219]:
pd.DataFrame(results['ACCURACY', 'SCORE'].describe())

,ACCURACY
,SCORE
count,274.000000
mean,0.915537
std,0.114775
min,0.428571
25%,0.857143
50%,1.000000
75%,1.000000
max,1.000000


### Filter out the wrong predictions from the table to calculate the weighted average of our accurate prediction

In [220]:
log = (tester_log == deepeye_log).stack()
df = log[log]
df.index.names = ['FRAME_ID', 'FEATURE']
right_predictions = pd.DataFrame(df, index=df.index)

### Total of correct predictions by feature

In [221]:
right_predictions = right_predictions.groupby(['FEATURE']).size().reset_index(name='COUNT')
right_predictions

,FEATURE,COUNT
0,FRAME_ID,274
1,PEDESTRIAN,253
2,VEHICLES,254
3,BIKES,264
4,STOP_SIGN,274
5,TRAFFIC_LIGHT,243
6,OFF_LANE,202
7,COLLISION,266


### Predictions results for Pedstrians

In [222]:
target = results.groupby(results['PEDESTRIAN', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(PEDESTRIAN, PREDICTION)",,,,,,,,
0,230.0,0.937267,0.097490,0.428571,0.857143,1.000000,1.000000,1.0
1,44.0,0.801948,0.131495,0.428571,0.714286,0.857143,0.857143,1.0


In [223]:
tp = int(right_predictions[right_predictions['FEATURE'] == 'PEDESTRIAN']['COUNT'])
total = int(right_predictions[right_predictions['FEATURE'] == 'FRAME_ID']['COUNT'])
print('Accuracy: ', tp / total)

Accuracy:  0.9233576642335767


### Predictions results for Vehicles

In [224]:
target = results.groupby(results['VEHICLES', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(VEHICLES, PREDICTION)",,,,,,,,
0,4.0,0.821429,0.214286,0.571429,0.678571,0.857143,1.0,1.0
1,270.0,0.916931,0.112798,0.428571,0.857143,1.000000,1.0,1.0


In [225]:
tp = int(right_predictions[right_predictions['FEATURE'] == 'VEHICLES']['COUNT'])
total = int(right_predictions[right_predictions['FEATURE'] == 'FRAME_ID']['COUNT'])
print('Accuracy: ', tp / total)

Accuracy:  0.927007299270073


### Predictions results for Bikes

In [226]:
target = results.groupby(results['BIKES', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(BIKES, PREDICTION)",,,,,,,,
0,258.0,0.923034,0.110625,0.428571,0.857143,1.000000,1.000000,1.0
1,16.0,0.794643,0.116277,0.571429,0.714286,0.857143,0.857143,1.0


In [227]:
tp = int(right_predictions[right_predictions['FEATURE'] == 'BIKES']['COUNT'])
total = int(right_predictions[right_predictions['FEATURE'] == 'FRAME_ID']['COUNT'])
print('Accuracy: ', tp / total)

Accuracy:  0.9635036496350365


### Predictions results for Stop Signs

In [228]:
target = results.groupby(results['STOP_SIGN', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(STOP_SIGN, PREDICTION)",,,,,,,,
0,274.0,0.915537,0.114775,0.428571,0.857143,1.0,1.0,1.0


In [229]:
tp = int(right_predictions[right_predictions['FEATURE'] == 'STOP_SIGN']['COUNT'])
total = int(right_predictions[right_predictions['FEATURE'] == 'FRAME_ID']['COUNT'])
print('Accuracy: ', tp / total)

Accuracy:  1.0


### Predictions results for Traffic Lights

In [230]:
target = results.groupby(results['TRAFFIC_LIGHT', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(TRAFFIC_LIGHT, PREDICTION)",,,,,,,,
0,225.0,0.935238,0.093405,0.571429,0.857143,1.000000,1.0,1.0
1,49.0,0.825073,0.155033,0.428571,0.714286,0.857143,1.0,1.0


In [231]:
tp = int(right_predictions[right_predictions['FEATURE'] == 'TRAFFIC_LIGHT']['COUNT'])
total = int(right_predictions[right_predictions['FEATURE'] == 'FRAME_ID']['COUNT'])
print('Accuracy: ', tp / total)

Accuracy:  0.8868613138686131


### Predictions results for Off-Lane Warnings

In [232]:
target = results.groupby(results['OFF_LANE', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(OFF_LANE, PREDICTION)",,,,,,,,
0,262.0,0.918212,0.113283,0.428571,0.857143,1.000000,1.0,1.0
1,12.0,0.857143,0.136209,0.571429,0.821429,0.857143,1.0,1.0


In [233]:
tp = int(right_predictions[right_predictions['FEATURE'] == 'OFF_LANE']['COUNT'])
total = int(right_predictions[right_predictions['FEATURE'] == 'FRAME_ID']['COUNT'])
print('Accuracy: ', tp / total)

Accuracy:  0.7372262773722628


### Predictions results for Collision Warnings

In [234]:
target = results.groupby(results['COLLISION', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(COLLISION, PREDICTION)",,,,,,,,
0,266.0,0.920516,0.109486,0.428571,0.857143,1.000000,1.000000,1.000000
1,8.0,0.750000,0.166424,0.428571,0.678571,0.857143,0.857143,0.857143


In [235]:
tp = int(right_predictions[right_predictions['FEATURE'] == 'COLLISION']['COUNT'])
total = int(right_predictions[right_predictions['FEATURE'] == 'FRAME_ID']['COUNT'])
print('Accuracy: ', tp / total)

Accuracy:  0.9708029197080292
